In [1]:
#!/usr/bin/python
# coding: utf-8

from datetime import date, datetime, timedelta
from predict_functions import *
from sklearn.externals import joblib
import pandas as pd
import pymongo as pym
import re

In [3]:
# les fonctions appelées par ce notebook se trouvent dans le fichier python predict_functions

# les paramètres
# date = datetime.strftime(datetime.utcnow() - timedelta(hours=24), '%Y-%m-%d')
# dates = [('2017-04-' + str(d)) if d > 9 else ('2017-04-0' + str(d)) for d in range(5, 0, -1)]
dates = ['2017-04-22', '2017-04-21']
fname = 'data/twitter_sentiments_21_au_22_.csv'

df_pred = pd.DataFrame()

# 1. Chargement du vocabulaire et du modèle
voca = pd.read_json('trained_dict.json').to_dict()[0]
clf = joblib.load('trained_logistic_regression.pkl')

other_politicians = ['valls', 'sarko', 'hollande']
candidates = {'macron': 'macron|emmanuel',
              'fillon': 'fillon',
              'hamon': 'hamon|benoit|benoît',
              'melenchon': 'melenchon|mélenchon|jlm',
              'le pen': 'le pen|lepen|mlp|marine',
              'nda': 'aignan|dupont|nda',
              'asselineau': 'asselineau|asselinau|aselineau',
              'lassalle': 'lassalle|lassale|lasalle|lasale',
              'poutou': 'poutou',
              'arthaud': 'arthaud',
              'cheminade': 'cheminade'
             }
stop_words = '|'.join([pol for pol in other_politicians])
    
for date in dates:
    try:
        # 2. Chargement des tweets a predire
        # dataframe contenant les tweets a predire dans une colonne 'text'
        print(30*'-')
        print('Chargement des tweets des candidats depuis MongoDB pour le {}...'.format(date))
        print(30*'-')
        df = extract_tweets(date, days=1, port=27017, limit=250000) # limite pour éviter les MemoryError

        if df.shape[0]:
            # on repère les tweets où plusieurs candidats sont cités
            df['other'] = df['text'].str.contains(stop_words, case=False) 

            # on repère les candidats contenus dans les tweets
            for candidate in candidates:
                df[candidate] = df['text'].str.contains(candidate, case=False)

            # filtrage des tweets contenant d'autres personnalités politiques"'
            df = df[df['other']==False]

            # filtrage des tweets contenant plusieurs des 5 candidats (ou aucun candidat)
            df['count'] = 1 * df.fillon + df.macron + df['le pen'] + df.melenchon + df.hamon
            df = df[df['count']==1]
            df.reset_index(drop=True, inplace=True)

            # 3. Creation des features et de la matrice TF-IDF pour la base test
            X_test = build_X(df, drop_dups=False, vocab=voca, min_df=3, n_grams=(1,1))

            # 4. Prediction
            print('Prediction des tweets...')
            y_pred = clf.predict(X_test)
            del X_test
            df['sentiment'] = y_pred
            
#             try:
#                 print('Insertion dans la base MongoDB "predicted"...')
#                 insert_in_mongo(df.drop(['other', 'count'], axis=1), port=27017)
#             except:
#                 print('Echec de l\'insertion dans la base MongoDB')

            # 5. Sauvegarder les predictions
            # ajout de la ligne du candidat dans le dataframe
            for candidate in candidates:
                curr_df = df[df[candidate]==True]
                taille = curr_df.shape[0]
                rec = {'count': taille, 'candidat': candidate, 'date': date}
                try:
                    rec['neg'] = curr_df[curr_df['sentiment']==-1].shape[0] / taille
                    rec['neu'] = curr_df[curr_df['sentiment']==0].shape[0] / taille
                    rec['pos'] = curr_df[curr_df['sentiment']==1].shape[0] / taille
                except:
                    # si aucun tweet pour le candidat courant n'est dans la base (cas pathologique)
                    rec['neg'], rec['neu'], rec['pos'] = ('-', '-', '-')

                df_pred = df_pred.append(rec, verify_integrity=False, ignore_index=True)
            
            del df
    except:
        print('Erreur pour la date {}'.format(date))
        del X_test
        del df
        continue
        
df_pred.set_index(['date', 'candidat'], drop=True, inplace=True)
df_pred.sort_index(axis=0, level=[0,1], inplace=True)
print(df_pred)

print('Sauvegarde des pourcentages par candidat dans un .csv : {}'.format(fname))
df_pred.to_csv(fname, index_label=('date','candidat'))

------------------------------
Chargement des tweets des candidats depuis MongoDB pour le 2017-04-22...
------------------------------
95845 tweets trouves.
Tagging des tweets en cours...
TreeTagger a renvoye 0 erreur(s).
Creation de la matrice de features...
Taille du vocabulaire : 4884
52249 documents vectorises.
Prediction des tweets...
------------------------------
Chargement des tweets des candidats depuis MongoDB pour le 2017-04-21...
------------------------------
167605 tweets trouves.
Tagging des tweets en cours...
TreeTagger a renvoye 0 erreur(s).
Creation de la matrice de features...
Taille du vocabulaire : 4884
98233 documents vectorises.
Prediction des tweets...
                         count       neg       neu       pos
date       candidat                                         
2017-04-21 arthaud       103.0  0.000000  0.980583  0.019417
           asselineau    424.0  0.162736  0.806604  0.030660
           cheminade      63.0  0.380952  0.619048  0.000000
          

In [4]:
csv_file = pd.DataFrame(pd.read_csv(fname, sep = ',', header = 0, index_col = False))
csv_file.to_json('twitter_sentiments_daily_21_until_22_april_raw.json', orient ='split')
df = pd.read_json('twitter_sentiments_daily_21_until_22_april_raw.json', orient='split')
df['neu'] = pd.to_numeric(df['neu'], errors='coerce')
df['neg'] = pd.to_numeric(df['neg'], errors='coerce')
df['pos'] = pd.to_numeric(df['pos'], errors='coerce')
df = df.fillna(0)
df['score'] = (df['pos']) / (df['neg'] + df['neu'] + df['pos'])
df = df.fillna(0)
df['score'] = df['score'] * df['count']
df_pivot = df.pivot_table(index='date',columns='candidat',values='score').fillna(0)
df_pivot.to_json('twitter_sentiments_daily_21_until_22_april.json', orient='split')